In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Fri Aug  8 11:23:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# Install core libraries for the RAG prototype.
# -qU = quiet output + upgrade to latest compatible versions
# langchain            : framework for chaining LLM + retriever steps
# langchain-community  : community loaders/vector stores (e.g., DirectoryLoader, FAISS)
# langchain-openai     : OpenAI wrappers (ChatOpenAI, OpenAIEmbeddings)
# tiktoken             : OpenAI tokenizer for token counting/chunking
!pip install -qU langchain langchain-community langchain-openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [4]:
# Install FAISS with GPU
# Use only if your Colab runtime has an NVIDIA GPU (check using command "!nvidia-smi").
# -qU = quiet + upgrade. If this fails (CUDA mismatch/no GPU), use "pip install -qU faiss-cpu"
!pip install -qU faiss-gpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [5]:
# Using CPU, as GPU command doesn't work.
!pip install -qU faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.9 MB/s eta 0:00:00


In [7]:
# Verify FAISS is installed and importable.
# This confirms a CPU-only install.
import faiss
print("FAISS OK (CPU)")

FAISS OK (CPU)
